## Calibrate AD99 parameters to check deficiency cannot be corrected
### Load data


In [1]:
import xarray as xr 
BASIN = None 
FIGDIR = '../../figures'
DATADIR = '../../data'
exp = 'AD99 c0=u0 (include unbroken)'
expcode = 'always_include_unbroken_1year'

ERA5_DATASET = f"/scratch/users/robcking/era5_{expcode}.nc"
GCM_DATASET = f"/scratch/users/robcking/mima_gwf_{expcode}.nc"

fluxes = ['gw_flux_eastward','gw_flux_westward','gw_flux_northward','gw_flux_southward','z']

ds_era5 = xr.open_dataset(ERA5_DATASET)
ds_gcm = xr.open_dataset(GCM_DATASET)

ds_era5 = ds_era5[fluxes]
ds_gcm = ds_gcm[fluxes]

ds_era5 = mask_dataset(ds_era5,dir=DATADIR,basins=BASIN)
ds_gcm = mask_dataset(ds_gcm,dir=DATADIR,basins=BASIN)

ds_era5 = ds_era5.transpose('time','points','level')
ds_gcm = ds_gcm.transpose('time','points','level')

NameError: name 'mask_dataset' is not defined

### $B_t$ / $F_{S0}$ - intermittency factor
This parameter sets $\epsilon$; the intermittency rate of GWs. As discussed in the manuscript this parameter can only shift the entire distribution by a multiplicative factor. In log-space this correpsonds to an additive shift. Thus we can always correct the mean (first moment of the distribution) BUT NOT the variance or the skew!